In [1]:
%cd D:\YeqinHUANG_Collection\ucl\week12_14.Jan.2024\text

D:\YeqinHUANG_Collection\ucl\week12_14.Jan.2024\text


# Build library

# Vectorise Text - 1

In [2]:
import os
import re
import ebooklib
from ebooklib import epub

In [3]:
import os
import re
import ebooklib
from ebooklib import epub

def preprocess(epub_file, book_id):
    def read_epub_paragraphs(epub_file, book_id):
        book = epub.read_epub(epub_file)
        paragraphs = []
        paragraph_nr = 1

        for item in book.get_items():
            if item.get_type() == ebooklib.ITEM_DOCUMENT:
                content = item.get_content().decode('utf-8')
                content = re.sub('<[^<]+?>', '', content)  
                content = re.sub('\s+', ' ', content) 
                content = re.sub('\n', ' ', content)
                content = content.strip()  
                content_paragraphs = content.split('.')
                merged_paragraph = ""
                
                for paragraph in content_paragraphs:
                    paragraph = paragraph.strip()
                    if len(paragraph) > 50:
                        if len(merged_paragraph) + len(paragraph) <= max_length:
                            merged_paragraph += paragraph + ". "
                        else:
                            if merged_paragraph:
                                paragraph_dict = {'paragraph': merged_paragraph.strip(), 'bookID': book_id, 'nr': paragraph_nr}
                                paragraphs.append(paragraph_dict)
                                paragraph_nr += 1
                            merged_paragraph = paragraph + ". "
                
                if merged_paragraph:
                    paragraph_dict = {'paragraph': merged_paragraph.strip(), 'bookID': book_id, 'nr': paragraph_nr}
                    paragraphs.append(paragraph_dict)
                    paragraph_nr += 1

        return paragraphs

    full_text = read_epub_paragraphs(epub_file, book_id)
    return full_text

def process_all_epub_files(folder_path):
    processed_data_list = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.epub'):
            epub_file = os.path.join(folder_path, filename)
            book_id = os.path.splitext(filename)[0]
            paragraphs = preprocess(epub_file, book_id)
            processed_data_list.extend(paragraphs)
            print(f"Processed: {epub_file}")

    return processed_data_list

folder_path = 'epub'
max_length = 1000  
processed_data_list = process_all_epub_files(folder_path)
print(f"Total paragraphs processed: {len(processed_data_list)}")



Processed: epub\casanova-casanova-part-27-expelled-from-spain.epub
Total paragraphs processed: 241


C:\Users\76392\anaconda3\envs\py38\lib\site-packages\ebooklib\epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')


In [4]:
processed_data_list

[{'paragraph': 'Casanova: Part 27 - Expelled From Spain Table of Contents Chapter I Chapter II Chapter III Chapter IV Chapter V Chapter VI.',
  'bookID': 'casanova-casanova-part-27-expelled-from-spain',
  'nr': 1},
 {'paragraph': 'Casanova: Part 27 - Expelled From Spain The Memoirs of Jacques Casanova De Seingalt Giacomo Casanova Translator : Arthur Machen Copyright © [#&lt;Date: 2017-11-02 ((2458060j,0s,0n),+0s,2299161j)&gt;] epubBooks All Rights Reserved. By payment of the required fees, you have been granted the non-exclusive, non-transferable right to access and read the text of this ebook on-screen or via personal text-to-speech computer systems. No part of this text may be reproduced, transmitted, downloaded, decompiled, reverse engineered, stored in or introduced into any information storage and retrieval system, in any form or by any means, whether electronic or mechanical, now known or hereinafter invented, without the express written permission of epubBooks.',
  'bookID': 'ca

In [6]:
processed_data_list[240]

{'paragraph': '" Again Betty threw herself on her knees, begging him to calm himself, as I was her preserver not her betrayer. "What do you mean by \'preserver\'?" said B― M― Betty gave him the letter, saying,— "Read that. " The Englishman read the letter through without rising from the ground, and as I was certain of its effect I opened the door and told the landlord to send his people away, and to get dinner for three, as everything had been settled.',
 'bookID': 'casanova-casanova-part-27-expelled-from-spain',
 'nr': 241}

In [37]:
para = [data['paragraph'] for data in processed_data_list]

In [38]:
para

['Casanova: Part 27 - Expelled From Spain Table of Contents Chapter I Chapter II Chapter III Chapter IV Chapter V Chapter VI.',
 'Casanova: Part 27 - Expelled From Spain The Memoirs of Jacques Casanova De Seingalt Giacomo Casanova Translator : Arthur Machen Copyright © [#&lt;Date: 2017-11-02 ((2458060j,0s,0n),+0s,2299161j)&gt;] epubBooks All Rights Reserved. By payment of the required fees, you have been granted the non-exclusive, non-transferable right to access and read the text of this ebook on-screen or via personal text-to-speech computer systems. No part of this text may be reproduced, transmitted, downloaded, decompiled, reverse engineered, stored in or introduced into any information storage and retrieval system, in any form or by any means, whether electronic or mechanical, now known or hereinafter invented, without the express written permission of epubBooks.',
 'Chapter I I Make a Mistake and Manucci Becomes My Mortal Foe--his Vengeance--i Leave Madrid--saragossa--valentia--

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [40]:
vectorizer = TfidfVectorizer(min_df=2)

In [62]:
tfidf_matrix = vectorizer.fit_transform(para)

In [63]:
tfidf_matrix

<241x2293 sparse matrix of type '<class 'numpy.float64'>'
	with 22978 stored elements in Compressed Sparse Row format>

In [45]:
len(para)

241

In [46]:
vectorizer.get_feature_names_out()

array(['27', 'abandon', 'abandoned', ..., 'your', 'yourself', 'youth'],
      dtype=object)

In [47]:
len(vectorizer.get_feature_names_out())

2293

In [48]:
from sklearn.metrics.pairwise import cosine_similarity

In [49]:
query = 'temples and the moon with a man'
processedQuery = vectorizer.transform([query])[0]
print(processedQuery)

  (0, 2251)	0.4535989067147847
  (0, 2032)	0.36053110501559976
  (0, 1224)	0.7316778717061222
  (0, 108)	0.3590443512979726


In [50]:
processedQuery = 'walk on the street with girlfriend'

In [51]:
query_vector = vectorizer.transform([processedQuery])

In [52]:
query_vector

<1x2293 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [53]:
similarities = cosine_similarity(query_vector, tfidf_matrix)

In [54]:
similarities

array([[0.        , 0.02805261, 0.02834476, 0.03875525, 0.05666707,
        0.02306078, 0.0274741 , 0.03450195, 0.0339155 , 0.06149402,
        0.0314203 , 0.02735314, 0.0276859 , 0.04418181, 0.03095878,
        0.04710203, 0.02641562, 0.02847355, 0.04213006, 0.02061846,
        0.04830005, 0.04582399, 0.06547552, 0.06867958, 0.03911242,
        0.12721003, 0.04701894, 0.05490601, 0.04682203, 0.03963062,
        0.03949716, 0.03678833, 0.05574733, 0.04027547, 0.03896819,
        0.04566595, 0.04880957, 0.04841096, 0.04135612, 0.05414521,
        0.05460411, 0.0346421 , 0.03522031, 0.04554015, 0.02239988,
        0.02805679, 0.06131212, 0.01183775, 0.01823952, 0.01836544,
        0.03752808, 0.05706323, 0.02091818, 0.04997594, 0.04834652,
        0.03394086, 0.02784328, 0.04107041, 0.05752275, 0.23306992,
        0.04660031, 0.02978591, 0.0336294 , 0.0757603 , 0.03332445,
        0.05080197, 0.03924743, 0.05865861, 0.05470022, 0.02010747,
        0.0345013 , 0.05976719, 0.04954388, 0.06

In [55]:
len(similarities[0])

241

In [56]:
nearest_neighbor_index = similarities.argmax()
nearest_neighbor = tfidf_matrix[nearest_neighbor_index]
similarity_score = similarities[0, nearest_neighbor_index]

In [57]:
nearest_neighbor_index, similarity_score

(59, 0.23306992431753754)

In [61]:
index = 59

if 0 <= index < len(processed_data_list):
    paragraph_info = processed_data_list[index]
    book_id = paragraph_info['bookID']
    nr = paragraph_info['nr']
    paragraph = paragraph_info['paragraph']

    print(f"Book ID: {book_id}, Paragraph Number: {nr}, Paragraph: {paragraph}")
else:
    print("Invalid index")

Book ID: casanova-casanova-part-27-expelled-from-spain, Paragraph Number: 60, Paragraph: " She called her sister, and told her to order the Genoese to leave the house and never to enter it again. The thing was 'done in a moment, but the sister told us that as he went out he had said,— "Se ne pentira. I occupied an hour in relating some of the injuries I had received from this scoundrelly fellow. The next day (November 15th), I went to Nina at the usual time, and after spending two hours in pleasant converse with her and her sister I went out as the clocks were striking midnight. The door of the house was under an arcade, which extended to the end of the street. It was a dark night; and I had scarcely gone twenty–five paces when two men suddenly rushed at me. I stepped back, drawing my sword, and exclaiming, "Assassins!" and then with a rapid movement, I thrust my blade into the body of the nearest assailant. I then left the arcade, and began to run down the street. The second assassin 

In [64]:
top_n = 5  
nearest_neighbor_indices = similarities.argsort()[0, ::-1][:top_n]

for idx in nearest_neighbor_indices:
    paragraph_info = processed_data_list[idx]
    book_id = paragraph_info['bookID']
    nr = paragraph_info['nr']
    paragraph = paragraph_info['paragraph']
    similarity_score = similarities[0, idx]

    print(f"Book ID: {book_id}, Paragraph Number: {nr}, Similarity Score: {similarity_score:.2f}, Paragraph: {paragraph}")

Book ID: casanova-casanova-part-27-expelled-from-spain, Paragraph Number: 60, Similarity Score: 0.23, Paragraph: " She called her sister, and told her to order the Genoese to leave the house and never to enter it again. The thing was 'done in a moment, but the sister told us that as he went out he had said,— "Se ne pentira. I occupied an hour in relating some of the injuries I had received from this scoundrelly fellow. The next day (November 15th), I went to Nina at the usual time, and after spending two hours in pleasant converse with her and her sister I went out as the clocks were striking midnight. The door of the house was under an arcade, which extended to the end of the street. It was a dark night; and I had scarcely gone twenty–five paces when two men suddenly rushed at me. I stepped back, drawing my sword, and exclaiming, "Assassins!" and then with a rapid movement, I thrust my blade into the body of the nearest assailant. I then left the arcade, and began to run down the stre

# Reduce dimensionality

In [28]:
import numpy as np
from scipy.sparse import random
from sklearn.decomposition import TruncatedSVD

In [29]:
n_components = 250  
svd = TruncatedSVD(n_components=n_components, algorithm='randomized')
reduced_matrix = svd.fit_transform(tfidf_matrix)

In [30]:
reduced_query_vec = svd.transform(query_vector)

In [31]:
len(reduced_query_vec[0])

241

In [32]:
similarities2 = cosine_similarity(reduced_query_vec, reduced_matrix)

In [33]:
nearest_neighbor_index = similarities2.argmax()
nearest_neighbor = tfidf_matrix[nearest_neighbor_index]
similarity_score = similarities2[0, nearest_neighbor_index]

In [34]:
nearest_neighbor_index

59

In [35]:
paragraphs[59]

'" She called her sister, and told her to order the Genoese to leave the house and never to enter it again. The thing was \'done in a moment, but the sister told us that as he went out he had said,— "Se ne pentira. I occupied an hour in relating some of the injuries I had received from this scoundrelly fellow. The next day (November 15th), I went to Nina at the usual time, and after spending two hours in pleasant converse with her and her sister I went out as the clocks were striking midnight. The door of the house was under an arcade, which extended to the end of the street. It was a dark night; and I had scarcely gone twenty–five paces when two men suddenly rushed at me. I stepped back, drawing my sword, and exclaiming, "Assassins!" and then with a rapid movement, I thrust my blade into the body of the nearest assailant. I then left the arcade, and began to run down the street. The second assassin fired a pistol at me, but it fortunately missed me.'

# Vectorise Text - 2

In [75]:
from nltk.corpus import words
import gensim
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import words, stopwords, names

In [84]:
def get_paragraphs(processed_data_list):
    paras = []
    for data in processed_data_list:
        paras.append(data['paragraph'])
    return paras


In [85]:
from nltk.corpus import words
english_words_set = set(words.words())

In [86]:
def process_para(paras):
    para2 = []
    
    for p in paras:
        words = gensim.utils.simple_preprocess(p, min_len=3, deacc=True)

        lemmatizer = WordNetLemmatizer()
        lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

        stop_words = set(stopwords.words("english"))
        filtered_words = [word for word in lemmatized_words if (word not in stop_words) and (word in english_words_set)]

        stemmer = PorterStemmer()
        stemmed_words = [stemmer.stem(word) for word in filtered_words]

        processed_doc = " ".join(stemmed_words)

        para2.append(processed_doc)

    return para2

In [87]:
paras = get_paragraphs(processed_data_list)
para2 = process_para(para)

In [88]:
para2

['part tabl content chapter chapter chapter chapter chapter chapter',
 'part memoir translat copyright date right reserv payment fee non exclus non transfer right access read text screen via person text speech comput system part text may revers inform storag retriev system form mean whether electron mechan known hereinaft without express written permiss',
 'chapter make mistak becom mortal foe vengeanc leav arriv barcelona memoir written consol dread weari slowli kill perhap would kill anywher sinc though bodi old spirit desir young ever memoir ever read repeat read gone censur lost nevertheless see men divid two section one far greater compos ignor superfici learn reflect beg state latter would appeal judgment believ verac inde veraci man object deceiv chiefli write',
 'hitherto spoken noth truth without consid whether truth book work dogmat theolog think harm anyon fanci know imit bee get honey everi flower abl extract good catalogu vice virtu digress may long busi none must confess 

In [89]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse import random
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [90]:
def get_three_best_results(para2,all_paras,query):
    vectorizer = TfidfVectorizer(min_df=3)
    tfidf_matrix = vectorizer.fit_transform(para2)
    
    processedQuery = process_para([query])  
    query_vector = vectorizer.transform(processedQuery)
    similarities = cosine_similarity(query_vector, tfidf_matrix)
    nearest_neighbor_index = similarities.argmax()
    
    n_components = 250
    svd = TruncatedSVD(n_components=n_components, algorithm='randomized')
    reduced_matrix = svd.fit_transform(tfidf_matrix)
    reduced_query_vec = svd.transform(query_vector)
    similarities2 = cosine_similarity(reduced_query_vec, reduced_matrix)
    nearest_neighbor_index2 = similarities2.argmax()
    
    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(para2)]
    model = Doc2Vec(vector_size=250, min_count=0, alpha=0.025, min_alpha=0.025, epochs=100)
    model.build_vocab(documents)
    model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)
    
    if isinstance(processedQuery, str):
        processedQuery = [processedQuery]
    
    vector = model.infer_vector(processedQuery)
    nearest_neighbor_index3 = model.dv.most_similar(positive=[vector])[0][0]

    return [
        'TF-IDF: ' + all_paras[int(nearest_neighbor_index)]['paragraph'],
        'nr:' + str(all_paras[int(nearest_neighbor_index)]['nr']),
        'ID: ' + all_paras[int(nearest_neighbor_index)]['bookID'],
        'SVD: ' + all_paras[int(nearest_neighbor_index2)]['paragraph'],
        'nr:' + str(all_paras[int(nearest_neighbor_index2)]['nr']),
        'ID: ' + all_paras[int(nearest_neighbor_index2)]['bookID'],
        'Doc2Vec: ' + all_paras[int(nearest_neighbor_index3)]['paragraph'],
        'nr:' + str(all_paras[int(nearest_neighbor_index3)]['nr']),
        'ID: ' + all_paras[int(nearest_neighbor_index3)]['bookID'],
    ]

In [92]:
get_three_best_results(para2,processed_data_list,'Tall white pyramids reaching for the sky')

['TF-IDF: She started in the morning begging me to wait till the evening, and to travel by night so as to reach Barcelona by day–time. She told me to put up at the "Santa Maria," and not to call till I had heard from her. I followed all the directions given me by this curious woman, and found myself comfortably lodged at Barcelona. My landlord was a Swiss who told me in confidence that he had received instructions to treat me well, and that I had only to ask for what I wanted.',
 'nr:49',
 'ID: casanova-casanova-part-27-expelled-from-spain',
 'SVD: She started in the morning begging me to wait till the evening, and to travel by night so as to reach Barcelona by day–time. She told me to put up at the "Santa Maria," and not to call till I had heard from her. I followed all the directions given me by this curious woman, and found myself comfortably lodged at Barcelona. My landlord was a Swiss who told me in confidence that he had received instructions to treat me well, and that I had only

In [96]:
para

['Casanova: Part 27 - Expelled From Spain Table of Contents Chapter I Chapter II Chapter III Chapter IV Chapter V Chapter VI.',
 'Casanova: Part 27 - Expelled From Spain The Memoirs of Jacques Casanova De Seingalt Giacomo Casanova Translator : Arthur Machen Copyright © [#&lt;Date: 2017-11-02 ((2458060j,0s,0n),+0s,2299161j)&gt;] epubBooks All Rights Reserved. By payment of the required fees, you have been granted the non-exclusive, non-transferable right to access and read the text of this ebook on-screen or via personal text-to-speech computer systems. No part of this text may be reproduced, transmitted, downloaded, decompiled, reverse engineered, stored in or introduced into any information storage and retrieval system, in any form or by any means, whether electronic or mechanical, now known or hereinafter invented, without the express written permission of epubBooks.',
 'Chapter I I Make a Mistake and Manucci Becomes My Mortal Foe--his Vengeance--i Leave Madrid--saragossa--valentia--

In [97]:
from gensim.utils import simple_preprocess
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk

# Only need to download these once, ideally at the start of your script
nltk.download('wordnet')
nltk.download('stopwords')

def preprocess_text(para):
    # Ensure para is a string
    if isinstance(para, list):
        para = ' '.join(para)

    # Basic text processing
    words = simple_preprocess(para, deacc=True)

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

    # Removing stopwords
    stop_words = set(stopwords.words("english"))
    filtered_words = [word for word in lemmatized_words if word not in stop_words]

    return filtered_words

# Example use
para = ["Your first paragraph here.", "Your second paragraph here."]
processed_para = preprocess_text(para)

print(processed_para)


['first', 'paragraph', 'second', 'paragraph']


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\76392\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\76392\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [36]:
import gensim
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import nltk
from bs4 import BeautifulSoup
import ebooklib
from ebooklib import epub
import os

In [35]:
# def extract_text_from_epub(epub_path):
#     book = epub.read_epub(epub_path)
#     text = ''
#     for item in book.get_items():
#         if item.get_type() == ebooklib.ITEM_DOCUMENT:
#             soup = BeautifulSoup(item.get_content(), 'html.parser')
#             text += soup.get_text() + ' '
#     return text

# def preprocess2(text):
#     def clean_html_tags(text):
#         if isinstance(text, str):
#             soup = BeautifulSoup(text, 'html.parser')
#             cleaned_text = soup.get_text()
#             return cleaned_text
#         else:
#             return ''

#     cleaned_text = clean_html_tags(text)

#     words = gensim.utils.simple_preprocess(cleaned_text, min_len=2, deacc=True)

#     lemmatizer = WordNetLemmatizer()
#     lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

#     stop_words = set(stopwords.words("english"))
#     english_words = set(nltk.corpus.words.words())

#     def is_english_word(word):
#         return word in english_words

#     filtered_words = [word for word in lemmatized_words if word not in stop_words and is_english_word(word)]

#     return filtered_words

# def process_all_epub_files(folder_path):
#     all_processed_texts = []
#     total_files_processed = 0
#     total_words_processed = 0

#     for filename in os.listdir(folder_path):
#         if filename.endswith('.epub'):
#             epub_file = os.path.join(folder_path, filename)
#             text = extract_text_from_epub(epub_file)
#             processed_text = preprocess2(text)
#             all_processed_texts.extend(processed_text)
#             total_files_processed += 1
#             total_words_processed += len(processed_text)

#     print(f"Total files processed: {total_files_processed}")
#     print(f"Total words processed: {total_words_processed}")

#     return all_processed_texts

# folder_path = 'epub'
# processed_texts = process_all_epub_files(folder_path)

# for text in processed_texts:
#     print(text)

C:\Users\76392\anaconda3\envs\py38\lib\site-packages\ebooklib\epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')


Total files processed: 1
Total words processed: 17005
part
table
content
chapter
chapter
chapter
chapter
chapter
chapter
part
memoir
de
translator
copyright
right
reserved
publication
copyright
payment
fee
non
exclusive
non
transferable
right
access
read
text
screen
via
personal
text
speech
computer
system
part
text
may
reverse
information
storage
retrieval
system
form
mean
whether
electronic
mechanical
known
hereinafter
without
express
written
permission
chapter
make
mistake
becomes
mortal
foe
vengeance
leave
arrive
barcelona
memoir
written
console
dreadful
weariness
slowly
killing
perhaps
would
kill
anywhere
since
though
body
old
spirit
desire
young
ever
memoir
ever
read
repeat
read
gone
censure
lost
nevertheless
seeing
men
divided
two
section
one
far
greater
composed
ignorant
superficial
learned
reflective
beg
state
latter
would
appeal
judgment
believe
veracity
indeed
veracious
man
object
deceiving
chiefly
write
hitherto
spoken
nothing
truth
without
considering
whether
truth
book
wo

In [37]:
# import os
# from gensim.models.doc2vec import TaggedDocument, Doc2Vec

# folder_path = 'your_epub_folder'
# epub_files = [file for file in os.listdir(folder_path) if file.endswith('.epub')]

# # 准备一个列表来存储所有段落
# all_paragraphs = []

# # 读取每个EPUB文件并提取段落
# for i, epub_file in enumerate(epub_files):
#     book_id = f'book_{i}'  # 每本书都有一个唯一的ID
#     paragraphs = read_epub_paragraphs(os.path.join(folder_path, epub_file), book_id)
#     all_paragraphs.extend(paragraphs)

# # 创建标记文档以供模型训练
# documents = [TaggedDocument(words=preprocess2(p['paragraph']), tags=[p['bookID'], str(p['nr'])]) for p in all_paragraphs]

# # 训练Doc2Vec模型
# model = Doc2Vec(vector_size=250, min_count=5, alpha=0.025, min_alpha=0.025, epochs=100)
# model.build_vocab(documents)
# model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

['wa', 'would', 'day', 'told', 'said', 'man', 'could', 'one', 'see', 'good', 'made', 'letter', 'ha', 'never', 'de', 'shall', 'must', 'count', 'know', 'came', 'go', 'gave', 'give', 'time', 'two', 'thought', 'went', 'love', 'without', 'make', 'may', 'take', 'might', 'left', 'betty', 'like', 'three', 'nothing', 'husband', 'think', 'come', 'well', 'year', 'much', 'though', 'tell', 'found', 'dinner', 'friend', 'next', 'woman', 'way', 'hour', 'going', 'great', 'order', 'saw', 'leave', 'say', 'landlord', 'house', 'done', 'received', 'lady', 'wife', 'took', 'room', 'sir', 'got', 'name', 'barcelona', 'ambassador', 'knew', 'town', 'money', 'till', 'young', 'viceroy', 'word', 'back', 'let', 'hope', 'sure', 'first', 'reason', 'taken', 'last', 'clock', 'best', 'hand', 'given', 'marquis', 'enough', 'evening', 'girl', 'sent', 'gone', 'away', 'week', 'fellow', 'officer', 'find', 'manner', 'pleasure', 'lover', 'inn', 'seen', 'every', 'doubt', 'long', 'want', 'little', 'poor', 'brought', 'old', 'thing',

In [54]:
# from gensim.models.doc2vec import TaggedDocument, Doc2Vec

# vector_size = 100  
# epochs = 20  

# documents = [TaggedDocument(words=preprocess2(text), tags=[i]) for i, text in enumerate(processed_texts)]
# model = Doc2Vec(vector_size=vector_size, min_count=5, epochs=epochs)

# model.build_vocab(documents)
# model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

# print(model.wv.index_to_key)


['wa', 'said', 'would', 'one', 'king', 'man', 'could', 'day', 'hand', 'came', 'great', 'go', 'see', 'well', 'good', 'come', 'upon', 'men', 'time', 'went', 'like', 'know', 'ha', 'two', 'made', 'woman', 'back', 'must', 'never', 'shall', 'god', 'say', 'little', 'might', 'take', 'thought', 'give', 'way', 'away', 'still', 'hornblower', 'let', 'took', 'life', 'eye', 'love', 'make', 'de', 'saw', 'people', 'even', 'face', 'father', 'told', 'many', 'last', 'first', 'night', 'may', 'lord', 'tell', 'much', 'head', 'right', 'heart', 'every', 'long', 'word', 'left', 'think', 'friend', 'though', 'nothing', 'old', 'gave', 'three', 'done', 'thing', 'death', 'city', 'put', 'without', 'hour', 'till', 'lady', 'get', 'knew', 'side', 'place', 'look', 'house', 'yet', 'young', 'year', 'horse', 'another', 'moment', 'set', 'sir', 'son', 'arm', 'also', 'ever', 'room', 'round', 'name', 'prince', 'found', 'town', 'wife', 'going', 'dead', 'sent', 'count', 'stood', 'end', 'hundred', 'child', 'find', 'far', 'foot', 

In [38]:
# test = model.predict_output_word(['old', 'house', 'come', 'woman'])
# print(test)

[('old', 0.0012924559), ('duke', 0.0012924136), ('manner', 0.0012923533), ('word', 0.001292318), ('much', 0.0012923107), ('yes', 0.0012922949), ('bed', 0.0012922902), ('next', 0.0012922898), ('gave', 0.001292288), ('money', 0.0012922874)]


In [39]:
# query = 'a man thinking about his life under the full moon'
# tokens = preprocess2(query)
# vector = model.infer_vector(tokens)


In [40]:
# vector

array([-0.00362064,  0.0026488 , -0.00313573, -0.0002655 ,  0.00245586,
        0.0019807 , -0.00061382,  0.00317313, -0.00226465, -0.0072374 ,
       -0.00092191,  0.010501  ,  0.00122655,  0.00354118,  0.00470485,
       -0.00067899,  0.00687836, -0.00233466,  0.00316508,  0.00605305,
        0.01550204,  0.00210619,  0.0047718 , -0.00672842, -0.00256005,
       -0.00393826,  0.00389137, -0.00119954,  0.00147567, -0.00655638,
        0.0039153 , -0.00881033, -0.00988428,  0.0030275 , -0.00271894,
        0.00332082, -0.00187341, -0.00047824, -0.00076827, -0.00379275,
        0.00025025,  0.00787835,  0.00413555,  0.00357257, -0.010001  ,
       -0.00433342,  0.00046582, -0.00633515, -0.0005255 ,  0.00722102,
       -0.00039623, -0.00139082, -0.00182512,  0.00450722, -0.00123205,
        0.00428337, -0.00459022, -0.00109114, -0.00286139,  0.00086239,
        0.00375356,  0.00319569,  0.00144868, -0.00972395,  0.00803202,
        0.00864413,  0.00256686, -0.00393912,  0.00574089, -0.00

In [41]:
# model.dv.most_similar(positive=[vector])

[('1404', 0.4013587534427643),
 ('1507', 0.39843931794166565),
 ('15526', 0.39195582270622253),
 ('13238', 0.38410937786102295),
 ('3718', 0.38389068841934204),
 ('3854', 0.38102883100509644),
 ('347', 0.37036964297294617),
 ('13243', 0.36575794219970703),
 ('4542', 0.36017122864723206),
 ('16194', 0.35341572761535645)]

In [42]:
# paragraphs[1404]

IndexError: list index out of range

In [71]:
# paragraphs

['Casanova: Part 26 - Spanish Passions Table of Contents Chapter I Chapter II Chapter III Chapter IV Chapter V Chapter VI.',
 'Casanova: Part 26 - Spanish Passions The Memoirs of Jacques Casanova De Seingalt Giacomo Casanova Translator : Arthur Machen Copyright © 2017 epubBooks All Rights Reserved. By payment of the required fees, you have been granted the non-exclusive, non-transferable right to access and read the text of this ebook on-screen or via personal text-to-speech computer systems. No part of this text may be reproduced, transmitted, downloaded, decompiled, reverse engineered, stored in or introduced into any information storage and retrieval system, in any form or by any means, whether electronic or mechanical, now known or hereinafter invented, without the express written permission of epubBooks.',
 "Chapter I I Am Ordered to Leave Vienna--the Empress Moderates but Does Not Annul the Order--zavoiski at Munich--my Stay at Augsburg--gasconnade at Louisburg--the Cologne Newsp

In [47]:
    def read_epub_paragraphs(epub_file, book_id):
        book = epub.read_epub(epub_file)
        paragraphs = []
        paragraph_nr = 1

        for item in book.get_items():
            if item.get_type() == ebooklib.ITEM_DOCUMENT:
                content = item.get_content().decode('utf-8')
                content = re.sub('<[^<]+?>', '', content)  
                content = re.sub('\s+', ' ', content) 
                content = re.sub('\n', ' ', content)
                content = content.strip()  
                content_paragraphs = content.split('.')
                merged_paragraph = ""
                
                for paragraph in content_paragraphs:
                    paragraph = paragraph.strip()
                    if len(paragraph) > 50:
                        if len(merged_paragraph) + len(paragraph) <= max_length:
                            merged_paragraph += paragraph + ". "
                        else:
                            if merged_paragraph:
                                paragraph_dict = {'paragraph': merged_paragraph.strip(), 'bookID': book_id, 'nr': paragraph_nr}
                                paragraphs.append(paragraph_dict)
                                paragraph_nr += 1
                            merged_paragraph = paragraph + ". "
                
                if merged_paragraph:
                    paragraph_dict = {'paragraph': merged_paragraph.strip(), 'bookID': book_id, 'nr': paragraph_nr}
                    paragraphs.append(paragraph_dict)
                    paragraph_nr += 1

        return paragraphs


In [48]:
import os
from gensim.models.doc2vec import TaggedDocument, Doc2Vec

folder_path = 'epub'
epub_files = [file for file in os.listdir(folder_path) if file.endswith('.epub')]

# 准备一个列表来存储所有段落
all_paragraphs = []

# 读取每个EPUB文件并提取段落
for i, epub_file in enumerate(epub_files):
    book_id = f'book_{i}'  # 每本书都有一个唯一的ID
    paragraphs = read_epub_paragraphs(os.path.join(folder_path, epub_file), book_id)
    all_paragraphs.extend(paragraphs)

# 创建标记文档以供模型训练
documents = [TaggedDocument(words=preprocess2(p['paragraph']), tags=[p['bookID'], str(p['nr'])]) for p in all_paragraphs]

# 训练Doc2Vec模型
vector_size = 250  
epochs = 100  
model = Doc2Vec(vector_size=vector_size, min_count=5, epochs=epochs)
model.build_vocab(documents)
model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

# 查询最相似的段落
query = 'a man thinking about his life under the full moon'
tokens = preprocess2(query)
vector = model.infer_vector(tokens)
similar_paragraphs = model.dv.most_similar(positive=[vector])

# 打印最相似的段落序号
for paragraph_id, similarity in similar_paragraphs:
    paragraph_id = int(paragraph_id)
    if paragraph_id < len(all_paragraphs):
        similar_paragraph = all_paragraphs[paragraph_id]
        print(f"Similarity: {similarity}")
        print(f"Paragraph ID: {paragraph_id}")
        print(f"Paragraph: {similar_paragraph['paragraph']}")
        print()


ValueError: invalid literal for int() with base 10: 'book_0'